In [1]:
!pip install -q pyperclip
!pip install -q webdriver-manager
!pip install -q chromedriver-autoinstaller

In [11]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

import pyperclip
import time
import csv
import pandas as pd

from webdriver_manager.chrome import ChromeDriverManager
import chromedriver_autoinstaller  # setup chrome options

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
options.add_argument('--disable-gpu')
options.add_argument('lang=ko_KR') # 한국어

chromedriver_autoinstaller.install()  # set the target URL

'C:\\Users\\etukd\\anaconda3\\Lib\\site-packages\\chromedriver_autoinstaller\\125\\chromedriver.exe'

In [25]:
# 영화 데이터 크롤링
comments = {
    'date': [],
    'comment': [],
    'recomm': [],
    'unrecomm': []
}

In [26]:
base_url = "https://serieson.naver.com/v3/movie/products?tag=&sortType=POPULARITY_DESC&price=all" # 드라마
genre = 'comments'

In [39]:
driver = webdriver.Chrome(options=options)
driver.get(base_url)

time.sleep(3)

ID = 'etukdh6433' #본인ID
PW = 'tuticus643#' #본인PW

try:
    login_btn = driver.find_element(By.CSS_SELECTOR, '#gnb_login_button')
    login_btn.click()
    
    #driver.get("https://nid.naver.com/nidlogin.login")
    time.sleep(3)
    
    id = driver.find_element(By.CSS_SELECTOR, "#id")
    id.click()
    pyperclip.copy(ID)
    actions = ActionChains(driver)
    actions.key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()
    time.sleep(1)
    # id.send_keys(ID)
    
    pw = driver.find_element(By.CSS_SELECTOR, "#pw")
    pw.click()
    pyperclip.copy(PW)
    actions = ActionChains(driver)
    actions.key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()
    time.sleep(1)
    # pw.send_keys(PW)
    
    btn_login = driver.find_element(By.CSS_SELECTOR, "#log\.login")
    btn_login.click()
    
except Exception as e:
    print(f'Login Error Occurred: {e}')

finally:
    time.sleep(5)

In [40]:
# 더보기 클릭 (default = 1)
cnt_more = 61
for _ in range(1, cnt_more):
    try:
        more_button = driver.find_element(By.CSS_SELECTOR, 'button.ViewMoreButton_view_more_button__eGoJu')
        more_button.click()
        time.sleep(2)
    except:
        break

In [41]:
# 영화 갯수 => 반복문 실행 횟수
cnt_movie = cnt_more * 30 + 1

try:
    for i in range(931, cnt_movie):
        movie = driver.find_element(By.CSS_SELECTOR, f'#content > div.ListPage_category_wrap__L1R0q > ul > li:nth-child({i})')
        # 영화 세부 페이지로 이동
        movie.click()
        time.sleep(2)

        try:
            cb_btn = driver.find_element(By.CLASS_NAME, 'u_cbox_cleanbot_setbutton')
            cb_btn.click()

            cleanbot = driver.find_element(By.ID, 'cleanbot_dialog_checkbox_cbox_module').get_attribute('class')
            if cleanbot == 'u_cbox_layer_cleanbot2_checkbox is_checked':
                checkbox = driver.find_element(By.CLASS_NAME, 'u_cbox_layer_cleanbot2_checkwrap')
                checkbox.click()
            
            apply = driver.find_element(By.CLASS_NAME, 'u_cbox_layer_cleanbot2_extrabtn')
            apply.click()
            time.sleep(0.5)
        except:
            continue

        try:
            while True:
                try:
                    cbox_btn_more = driver.find_element(By.CSS_SELECTOR, 'a.u_cbox_btn_more')
                    cbox_btn_more.click()
                    time.sleep(0.5)
                except:
                    break
        except:
            continue

        com_list = driver.find_elements(By.XPATH, '//*[@id="cbox_module_wai_u_cbox_content_wrap_tabpanel"]/ul/li')

        for com in com_list:
            date = com.find_element(By.CLASS_NAME, 'u_cbox_date').get_attribute('data-value')
            comment = com.find_element(By.CLASS_NAME, 'u_cbox_contents').text
            recomm = com.find_element(By.CLASS_NAME, 'u_cbox_cnt_recomm').text
            unrecomm = com.find_element(By.CLASS_NAME, 'u_cbox_cnt_unrecomm').text

            # print([date, comment, recomm, unrecomm])

            # data 입력
            comments['date'].append(pd.to_datetime(date, format='%Y-%m-%dT%H:%M:%S%z').strftime('%Y-%m-%d %H:%M'))
            comments['comment'].append(comment)
            comments['recomm'].append(int(recomm))
            comments['unrecomm'].append(int(unrecomm))

        # 영화 리스트 페이지로 이동
        driver.back()
        time.sleep(3)
        # break
            
except Exception as e:
    print(f'Crawl Data Error Occurred: {e}')

finally:
    print(i)
    # driver.close()

1830


In [42]:
df = pd.DataFrame(comments)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31441 entries, 0 to 31440
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      31441 non-null  object
 1   comment   31441 non-null  object
 2   recomm    31441 non-null  int64 
 3   unrecomm  31441 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 982.7+ KB


In [43]:
# 변환된 데이터프레임 확인
df.head()

,date,comment,recomm,unrecomm
0,2023-08-15 23:32,네이버에서 8720원이래서 들어왔는데 왜 10900원이죠;,159,5
1,2023-08-12 13:18,"이럴꺼면 바로보기라고 하지 알라니까?,",63,11
2,2023-08-11 10:29,"배척과 차별의 핑계가 되었던 '다름'을 오히려 사랑하고, 하나가 되니... 따뜻하다...",47,5
3,2024-03-29 09:03,재생이 안 됩니다. 환불 부탁드립니다.,0,0
4,2024-03-01 20:06,재생이 안됩니다 환불 부탁드립니다,0,0


In [44]:
saved_com = pd.read_csv('movie_comments(19649).csv')

In [46]:
result = pd.concat([saved_com, df], ignore_index=True)
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51090 entries, 0 to 51089
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      51090 non-null  object
 1   comment   51090 non-null  object
 2   recomm    51090 non-null  int64 
 3   unrecomm  51090 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 1.6+ MB


In [47]:
result.head()

,date,comment,recomm,unrecomm
0,2023-08-15 23:32,네이버에서 8720원이래서 들어왔는데 왜 10900원이죠;,159,5
1,2023-08-12 13:18,"이럴꺼면 바로보기라고 하지 알라니까?,",63,11
2,2023-08-11 10:29,"배척과 차별의 핑계가 되었던 '다름'을 오히려 사랑하고, 하나가 되니... 따뜻하다...",47,5
3,2024-03-29 09:03,재생이 안 됩니다. 환불 부탁드립니다.,0,0
4,2024-03-01 20:06,재생이 안됩니다 환불 부탁드립니다,0,0


In [48]:
result.to_csv(f'movie_comments({len(result)}).csv', index=False)